In [2]:
# YouTube 다운로드 도구 설치
!pip install yt-dlp

print("✅ yt-dlp 설치 완료!")
import subprocess
import os

# YouTube URL
youtube_url = "https://www.youtube.com/shorts/vQfWixBn15U"

# 영상 다운로드
def download_video():
    print(f"📹 YouTube 영상 다운로드 시작...")
    print(f"🔗 URL: {youtube_url}")
    
    try:
        # yt-dlp 명령어 실행
        result = subprocess.run([
            'yt-dlp', 
            '-f', 'best[height<=720]',
            '-o', 'traffic_video.mp4',
            youtube_url
        ], capture_output=True, text=True)
        
        if result.returncode == 0:
            print("✅ 다운로드 성공!")
            return 'traffic_video.mp4'
        else:
            print(f"❌ 다운로드 실패: {result.stderr}")
            return None
    except Exception as e:
        print(f"❌ 오류: {e}")
        return None

# 다운로드 실행
video_file = download_video()

# 파일 확인
if video_file and os.path.exists(video_file):
    file_size = os.path.getsize(video_file) / (1024*1024)
    print(f"📁 파일 크기: {file_size:.1f} MB")
    print(f"🎯 파일명: {video_file}")
else:
    print("❌ 파일이 생성되지 않았습니다!")




[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
✅ yt-dlp 설치 완료!
📹 YouTube 영상 다운로드 시작...
🔗 URL: https://www.youtube.com/shorts/vQfWixBn15U
✅ 다운로드 성공!
📁 파일 크기: 0.9 MB
🎯 파일명: traffic_video.mp4


In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import detection
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time

print("✅ 라이브러리 import 완료!")


✅ 라이브러리 import 완료!


In [4]:
# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"💻 사용 중인 디바이스: {device}")
print(f"🎯 GPU: {torch.cuda.get_device_name(0)}")

# TrafficCamNet 스타일 모델 로드
model = detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.to(device)
model.eval()

print("✅ 모델 로드 완료!")


💻 사용 중인 디바이스: cuda
🎯 GPU: NVIDIA GeForce RTX 3090


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


✅ 모델 로드 완료!


In [5]:
def check_video():
    cap = cv2.VideoCapture(video_file)
    
    if not cap.isOpened():
        print("❌ 영상을 열 수 없습니다!")
        return None
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps
    
    print(f"📹 영상 정보:")
    print(f"  해상도: {width} x {height}")
    print(f"  FPS: {fps:.1f}")
    print(f"  총 프레임: {total_frames}")
    print(f"  길이: {duration:.1f}초")
    
    cap.release()
    return {'fps': fps, 'width': width, 'height': height, 'frames': total_frames}

# 영상 정보 확인
if video_file:
    video_info = check_video()


def check_video():
    cap = cv2.VideoCapture(video_file)
    
    if not cap.isOpened():
        print("❌ 영상을 열 수 없습니다!")
        return None
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps
    
    print(f"📹 영상 정보:")
    print(f"  해상도: {width} x {height}")
    print(f"  FPS: {fps:.1f}")
    print(f"  총 프레임: {total_frames}")
    print(f"  길이: {duration:.1f}초")
    
    cap.release()
    return {'fps': fps, 'width': width, 'height': height, 'frames': total_frames}

# 영상 정보 확인
if video_file:
    video_info = check_video()
def draw_detections(frame, results):
    annotated = frame.copy()
    
    # 차량 (빨간색)
    for vehicle in results['vehicles']:
        box = vehicle['box'].astype(int)
        cv2.rectangle(annotated, (box[0], box[1]), (box[2], box[3]), (0, 0, 255), 3)
        label = f"{vehicle['class_name']}: {vehicle['score']:.2f}"
        cv2.putText(annotated, label, (box[0], box[1]-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    # 보행자 (파란색)
    for person in results['pedestrians']:
        box = person['box'].astype(int)
        cv2.rectangle(annotated, (box[0], box[1]), (box[2], box[3]), (255, 0, 0), 3)
        label = f"Person: {person['score']:.2f}"
        cv2.putText(annotated, label, (box[0], box[1]-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
    
    # 교통 인프라 (초록색)
    for infra in results['infrastructure']:
        box = infra['box'].astype(int)
        cv2.rectangle(annotated, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 3)
        label = f"{infra['class_name']}: {infra['score']:.2f}"
        cv2.putText(annotated, label, (box[0], box[1]-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    return annotated

print("✅ 바운딩 박스 함수 정의 완료!")



📹 영상 정보:
  해상도: 360 x 640
  FPS: 30.0
  총 프레임: 372
  길이: 12.4초
📹 영상 정보:
  해상도: 360 x 640
  FPS: 30.0
  총 프레임: 372
  길이: 12.4초
✅ 바운딩 박스 함수 정의 완료!


In [6]:
def analyze_traffic_video():
    print(f"\n🚀 TrafficCamNet 영상 분석 시작!")
    print("=" * 50)
    
    # 영상 열기
    cap = cv2.VideoCapture(video_file)
    
    if not cap.isOpened():
        print("❌ 영상을 열 수 없습니다!")
        return
    
    # 영상 정보
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # 출력 영상 설정
    output_path = "analyzed_traffic.mp4"
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    # 통계 변수
    frame_count = 0
    vehicle_counts = []
    pedestrian_counts = []
    
    start_time = time.time()
    
    print(f"🎯 총 {total_frames}프레임 분석 시작...")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        
        # RGB 변환
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # TrafficCamNet 분석
        results = analyze_frame(frame_rgb, confidence_threshold=0.4)
        
        # 통계 수집
        vehicles = len(results['vehicles'])
        pedestrians = len(results['pedestrians'])
        infrastructure = len(results['infrastructure'])
        
        vehicle_counts.append(vehicles)
        pedestrian_counts.append(pedestrians)
        
        # 바운딩 박스 그리기
        annotated_frame = draw_detections(frame, results)
        
        # 정보 오버레이
        current_time = frame_count / fps
        info_lines = [
            f"Frame: {frame_count}/{total_frames}",
            f"Time: {current_time:.1f}s",
            f"Vehicles: {vehicles}",
            f"Pedestrians: {pedestrians}",
            f"Infrastructure: {infrastructure}",
            "RTX 3090 Analysis"
        ]
        
        # 배경 박스
        cv2.rectangle(annotated_frame, (10, 10), (300, 180), (0, 0, 0), -1)
        
        # 정보 텍스트
        y_pos = 35
        for line in info_lines:
            cv2.putText(annotated_frame, line, (20, y_pos), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            y_pos += 25
        
        # 출력 영상에 쓰기
        out.write(annotated_frame)
        
        # 진행 상황 (10프레임마다)
        if frame_count % 10 == 0:
            progress = frame_count / total_frames * 100
            elapsed = time.time() - start_time
            fps_current = frame_count / elapsed if elapsed > 0 else 0
            
            print(f"진행률: {progress:5.1f}% | "
                  f"차량: {vehicles:2d}대 | "
                  f"보행자: {pedestrians:2d}명 | "
                  f"속도: {fps_current:5.1f} FPS")
    
    # 정리
    cap.release()
    out.release()
    
    # 최종 결과
    end_time = time.time()
    total_time = end_time - start_time
    avg_fps = frame_count / total_time
    
    print("\n" + "=" * 50)
    print("🎬 YouTube TrafficCamNet 분석 완료!")
    print("=" * 50)
    print(f"⏱️ 총 처리 시간: {total_time:.1f}초")
    print(f"🔥 평균 처리 속도: {avg_fps:.1f} FPS")
    
    if vehicle_counts:
        print(f"🚗 최대 차량 수: {max(vehicle_counts)}대")
        print(f"🚗 평균 차량 수: {sum(vehicle_counts)/len(vehicle_counts):.1f}대")
    
    if pedestrian_counts:
        print(f"🚶 최대 보행자 수: {max(pedestrian_counts)}명")
        print(f"🚶 평균 보행자 수: {sum(pedestrian_counts)/len(pedestrian_counts):.1f}명")
    
    print(f"💾 결과 파일: {output_path}")
    
    return output_path

print("✅ 메인 분석 함수 정의 완료!")


✅ 메인 분석 함수 정의 완료!


In [13]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import detection
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
import os

print("✅ 라이브러리 import 완료!")

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"💻 디바이스: {device}")
print(f"🎯 GPU: {torch.cuda.get_device_name(0)}")

# 모델 로드
model = detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.to(device)
model.eval()
print("✅ TrafficCamNet 모델 준비 완료!")

✅ 라이브러리 import 완료!
💻 디바이스: cuda
🎯 GPU: NVIDIA GeForce RTX 3090
✅ TrafficCamNet 모델 준비 완료!


In [14]:
# 클래스 이름 변환
def get_class_name(class_id):
    class_names = {
        1: 'person', 2: 'bicycle', 3: 'car', 4: 'motorcycle',
        6: 'bus', 8: 'truck', 10: 'traffic_light', 13: 'stop_sign'
    }
    return class_names.get(class_id, f'object_{class_id}')

# 프레임 분석 함수
def analyze_frame(image, confidence_threshold=0.4):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor()
    ])
    
    input_tensor = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        predictions = model(input_tensor)
    
    scores = predictions[0]['scores'].cpu().numpy()
    labels = predictions[0]['labels'].cpu().numpy()
    boxes = predictions[0]['boxes'].cpu().numpy()
    
    vehicles = []
    pedestrians = []
    infrastructure = []
    
    VEHICLE_CLASSES = [2, 3, 4, 6, 8]
    
    for score, label, box in zip(scores, labels, boxes):
        if score > confidence_threshold:
            obj_info = {
                'class_name': get_class_name(label),
                'score': score,
                'box': box
            }
            
            if label in VEHICLE_CLASSES:
                vehicles.append(obj_info)
            elif label == 1:
                pedestrians.append(obj_info)
            elif label in [10, 13]:
                infrastructure.append(obj_info)
    
    return {
        'vehicles': vehicles,
        'pedestrians': pedestrians,
        'infrastructure': infrastructure
    }

# 바운딩 박스 그리기
def draw_detections(frame, results):
    annotated = frame.copy()
    
    # 차량 (빨간색)
    for vehicle in results['vehicles']:
        box = vehicle['box'].astype(int)
        cv2.rectangle(annotated, (box[0], box[1]), (box[2], box[3]), (0, 0, 255), 2)
        label = f"{vehicle['class_name']}: {vehicle['score']:.2f}"
        cv2.putText(annotated, label, (box[0], box[1]-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    
    # 보행자 (파란색)
    for person in results['pedestrians']:
        box = person['box'].astype(int)
        cv2.rectangle(annotated, (box[0], box[1]), (box[2], box[3]), (255, 0, 0), 2)
        label = f"Person: {person['score']:.2f}"
        cv2.putText(annotated, label, (box[0], box[1]-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
    # 교통 인프라 (초록색)
    for infra in results['infrastructure']:
        box = infra['box'].astype(int)
        cv2.rectangle(annotated, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
        label = f"{infra['class_name']}: {infra['score']:.2f}"
        cv2.putText(annotated, label, (box[0], box[1]-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    return annotated

print("✅ 모든 함수 정의 완료!")

✅ 모든 함수 정의 완료!


In [15]:
def analyze_traffic_video():
    """TrafficCamNet 영상 분석"""
    
    # 사용할 영상 파일 찾기
    video_files = []
    for file in os.listdir('.'):
        if file.endswith(('.mp4', '.avi', '.mov', '.mkv')):
            video_files.append(file)
    
    if not video_files:
        print("❌ 영상 파일이 없습니다!")
        return None
    
    # 첫 번째 영상 파일 사용
    video_path = video_files[0]
    print(f"🎯 사용할 영상: {video_path}")
    
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"❌ 영상을 열 수 없습니다: {video_path}")
        return None
    
    # 영상 정보
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"📹 영상 정보: {width}x{height}, {fps}fps, {total_frames}프레임")
    
    # 출력 영상 설정
    output_path = "traffic_analysis_result.mp4"
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    frame_count = 0
    vehicle_stats = []
    pedestrian_stats = []
    
    start_time = time.time()
    
    print("🚀 TrafficCamNet 분석 시작!")
    print("=" * 50)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        
        # RGB 변환
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # TrafficCamNet 분석
        results = analyze_frame(frame_rgb)
        
        # 통계 수집
        vehicles = len(results['vehicles'])
        pedestrians = len(results['pedestrians'])
        infrastructure = len(results['infrastructure'])
        
        vehicle_stats.append(vehicles)
        pedestrian_stats.append(pedestrians)
        
        # 바운딩 박스 그리기
        annotated_frame = draw_detections(frame, results)
        
        # 정보 오버레이
        current_time = frame_count / fps
        info_lines = [
            f"Frame: {frame_count}/{total_frames}",
            f"Time: {current_time:.1f}s",
            f"Vehicles: {vehicles}",
            f"Pedestrians: {pedestrians}",
            f"Infrastructure: {infrastructure}",
            "RTX 3090 TrafficCamNet"
        ]
        
        # 배경 박스
        cv2.rectangle(annotated_frame, (5, 5), (280, 160), (0, 0, 0), -1)
        
        # 정보 텍스트
        y_pos = 25
        for line in info_lines:
            cv2.putText(annotated_frame, line, (10, y_pos), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            y_pos += 22
        
        # 출력 영상에 쓰기
        out.write(annotated_frame)
        
        # 진행 상황 (10프레임마다)
        if frame_count % 10 == 0:
            progress = frame_count / total_frames * 100
            elapsed = time.time() - start_time
            fps_current = frame_count / elapsed if elapsed > 0 else 0
            
            print(f"진행률: {progress:5.1f}% | "
                  f"차량: {vehicles:2d}대 | "
                  f"보행자: {pedestrians:2d}명 | "
                  f"속도: {fps_current:5.1f} FPS")
    
    # 정리
    cap.release()
    out.release()
    
    # 최종 결과
    end_time = time.time()
    total_time = end_time - start_time
    
    print("\n" + "=" * 50)
    print("🎬 TrafficCamNet 분석 완료!")
    print("=" * 50)
    print(f"⏱️ 처리 시간: {total_time:.1f}초")
    print(f"🔥 평균 속도: {frame_count/total_time:.1f} FPS")
    
    if vehicle_stats:
        print(f"🚗 최대 차량: {max(vehicle_stats)}대")
        print(f"🚗 평균 차량: {sum(vehicle_stats)/len(vehicle_stats):.1f}대")
    
    if pedestrian_stats:
        print(f"🚶 최대 보행자: {max(pedestrian_stats)}명")
        print(f"🚶 평균 보행자: {sum(pedestrian_stats)/len(pedestrian_stats):.1f}명")
    
    if os.path.exists(output_path):
        output_size = os.path.getsize(output_path) / (1024*1024)
        print(f"💾 결과 파일: {output_path} ({output_size:.1f} MB)")
    
    return output_path

print("✅ 영상 분석 함수 준비 완료!")

✅ 영상 분석 함수 준비 완료!


In [16]:
# TrafficCamNet 영상 분석 실행
result_video = analyze_traffic_video()

if result_video:
    print(f"\n✅ 모든 분석 완료!")
    print(f"📹 결과 영상: {result_video}")
else:
    print("❌ 분석 실패!")

🎯 사용할 영상: traffic_short.mp4
📹 영상 정보: 360x640, 30fps, 372프레임
🚀 TrafficCamNet 분석 시작!
진행률:   2.7% | 차량: 21대 | 보행자: 23명 | 속도:   8.9 FPS
진행률:   5.4% | 차량: 15대 | 보행자: 25명 | 속도:  11.2 FPS
진행률:   8.1% | 차량: 12대 | 보행자: 24명 | 속도:  11.6 FPS
진행률:  10.8% | 차량: 16대 | 보행자: 23명 | 속도:  12.2 FPS
진행률:  13.4% | 차량:  8대 | 보행자: 24명 | 속도:  12.6 FPS
진행률:  16.1% | 차량: 11대 | 보행자: 24명 | 속도:  13.1 FPS
진행률:  18.8% | 차량: 10대 | 보행자: 18명 | 속도:  13.3 FPS
진행률:  21.5% | 차량:  6대 | 보행자: 20명 | 속도:  13.4 FPS
진행률:  24.2% | 차량: 15대 | 보행자: 24명 | 속도:  13.6 FPS
진행률:  26.9% | 차량: 16대 | 보행자: 20명 | 속도:  13.7 FPS
진행률:  29.6% | 차량: 11대 | 보행자: 18명 | 속도:  13.8 FPS
진행률:  32.3% | 차량: 12대 | 보행자: 19명 | 속도:  13.9 FPS
진행률:  34.9% | 차량:  8대 | 보행자: 21명 | 속도:  13.9 FPS
진행률:  37.6% | 차량: 21대 | 보행자: 26명 | 속도:  14.0 FPS
진행률:  40.3% | 차량: 12대 | 보행자: 18명 | 속도:  14.1 FPS
진행률:  43.0% | 차량: 12대 | 보행자: 14명 | 속도:  14.1 FPS
진행률:  45.7% | 차량:  9대 | 보행자: 10명 | 속도:  14.1 FPS
진행률:  48.4% | 차량:  7대 | 보행자: 14명 | 속도:  14.2 FPS
진행률:  51.1% | 차량: 16대 | 보행자: 14명 | 